In [1]:
from keras.layers.core import Activation, Dense

from keras.layers.embeddings import Embedding

from keras.layers.recurrent import LSTM

from keras.models import Sequential

from keras.preprocessing import sequence

from sklearn.model_selection import train_test_split

import nltk #用来分词

import collections #用来统计词频

import numpy as np

Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np

In [3]:
train_data = pd.read_csv('datasets/train.csv',lineterminator="\n")
train_data.head()

,ID,review,label
0,1,Jo bhi ap se tou behtar hoon,Negative
1,2,ya Allah meri sister Affia ki madad farma,Positive
2,3,Yeh khud chahta a is umar main shadi krna. ha...,Negative
3,4,Tc ? Apky mun xe exe alfax achy nae lgty 😒💃,Negative
4,5,Good,Positive


In [4]:
len(train_data)

6328

In [5]:
from keras.layers.core import Activation, Dense

from keras.layers.embeddings import Embedding

from keras.layers.recurrent import LSTM

from keras.models import Sequential

from keras.preprocessing import sequence

from sklearn.model_selection import train_test_split

import nltk #用来分词

import collections #用来统计词频

import numpy as np

In [6]:
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zlm31\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zlm31\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
maxlen = 0
num_recs = 0 #总样本数
word_freqs = collections.Counter() #
for index,each in train_data.iterrows():
    words = nltk.word_tokenize(each['review'].strip().lower())
    if len(words) > maxlen:
        maxlen = len(words)

    for word in words:
        word_freqs[word] += 1
    num_recs += 1
print('max_len ',maxlen)

print('nb_words ', len(word_freqs))

max_len  340
nb_words  18348


In [8]:
MAX_FEATURES = 15000

MAX_SENTENCE_LENGTH = 300

vocab_size = min(MAX_FEATURES, len(word_freqs)) + 2

word2index = {x[0]: i+2 for i, x in enumerate(word_freqs.most_common(MAX_FEATURES))}

word2index["PAD"] = 0

word2index["UNK"] = 1

index2word = {v:k for k, v in word2index.items()}
X = np.empty(num_recs,dtype=list)

y = np.zeros(num_recs)

i=0
for index,each in train_data.iterrows():
    words = nltk.word_tokenize(each['review'].strip().lower())
    seqs = []

    for word in words:

        if word in word2index:
            seqs.append(word2index[word])
        else:
            seqs.append(word2index["UNK"])
    X[i] = seqs
    if each['label']=='Positive':
         y[i] = 1
    else:
        y[i] = 0
    i += 1
X = sequence.pad_sequences(X, maxlen=MAX_SENTENCE_LENGTH)

In [9]:
len(X)

6328

In [10]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.1, random_state=42)

In [11]:
EMBEDDING_SIZE = 128

HIDDEN_LAYER_SIZE = 64

model = Sequential()

model.add(Embedding(vocab_size, EMBEDDING_SIZE,input_length=MAX_SENTENCE_LENGTH))

model.add(LSTM(HIDDEN_LAYER_SIZE, dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(1))

model.add(Activation("sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
BATCH_SIZE = 50

NUM_EPOCHS = 10

model.fit(Xtrain, ytrain, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,validation_data=(Xtest, ytest))

Instructions for updating:
Use tf.cast instead.
Train on 5695 samples, validate on 633 samples
Epoch 1/10
5695/5695 [==============================] - 34s 6ms/step - loss: 0.6317 - acc: 0.6363 - val_loss: 0.5054 - val_acc: 0.7551
Epoch 2/10
5695/5695 [==============================] - 34s 6ms/step - loss: 0.3608 - acc: 0.8442 - val_loss: 0.5059 - val_acc: 0.7551
Epoch 3/10
5695/5695 [==============================] - 34s 6ms/step - loss: 0.1725 - acc: 0.9391 - val_loss: 0.6375 - val_acc: 0.7378
Epoch 4/10
5695/5695 [==============================] - 34s 6ms/step - loss: 0.0782 - acc: 0.9733 - val_loss: 0.7589 - val_acc: 0.7583
Epoch 5/10
5695/5695 [==============================] - 34s 6ms/step - loss: 0.0496 - acc: 0.9849 - val_loss: 0.9157 - val_acc: 0.7393
Epoch 6/10
5695/5695 [==============================] - 34s 6ms/step - loss: 0.0281 - acc: 0.9903 - val_loss: 1.0652 - val_acc: 0.7362
Epoch 7/10
5695/5695 [==============================] - 34s 6ms/step - loss: 0.0183 - acc: 0.99

In [13]:
score, acc = model.evaluate(Xtest, ytest, batch_size=BATCH_SIZE)

print("nTest score: %.3f, accuracy: %.3f" % (score, acc))

633/633 [==============================] - 1s 1ms/step
nTest score: 1.414, accuracy: 0.730


In [14]:
data_test = pd.read_csv('datasets/test.csv',lineterminator="\n")

In [15]:
XX = np.empty(len(data_test),dtype=list)

i=0
for index,each in data_test.iterrows():
    words = nltk.word_tokenize(each['review'].strip().lower())
    seq = []

    for word in words:

        if word in word2index:

            seq.append(word2index[word])

        else:

            seq.append(word2index['UNK'])

    XX[i] = seq
    i+=1


XX = sequence.pad_sequences(XX, maxlen=MAX_SENTENCE_LENGTH)

In [16]:
Ypred2 = model.predict_proba(XX)

In [17]:
final = []

i=1;
for each in Ypred2:
    temp = []
    temp.append(i)
    i += 1;
    temp.append(float(each))
    final.append(temp)

In [18]:
name = ["ID","Pred"]
store_test = pd.DataFrame(columns=name,data=final)
store_test.to_csv('datasets/result.csv',index=False)

In [19]:
(Ypred2)

array([[0.99997175],
       [0.99996006],
       [0.09150803],
       ...,
       [0.5303735 ],
       [0.00752833],
       [0.0140788 ]], dtype=float32)

In [20]:
final

[[1, 0.9999717473983765],
 [2, 0.9999600648880005],
 [3, 0.09150803089141846],
 [4, 0.999990701675415],
 [5, 0.9994558095932007],
 [6, 0.9999852180480957],
 [7, 0.9997845888137817],
 [8, 0.001650303602218628],
 [9, 0.5862665772438049],
 [10, 0.0008812248706817627],
 [11, 0.9700109958648682],
 [12, 0.9227231740951538],
 [13, 0.0006006956100463867],
 [14, 0.9630899429321289],
 [15, 0.987240195274353],
 [16, 0.006054550409317017],
 [17, 0.0007216036319732666],
 [18, 0.9124009013175964],
 [19, 0.9995654821395874],
 [20, 0.999992847442627],
 [21, 0.5120435357093811],
 [22, 2.3692846298217773e-05],
 [23, 0.00023260712623596191],
 [24, 0.9999288320541382],
 [25, 0.0003510415554046631],
 [26, 0.00011104345321655273],
 [27, 0.0013169944286346436],
 [28, 0.999767005443573],
 [29, 0.00033867359161376953],
 [30, 0.9998586177825928],
 [31, 0.028111696243286133],
 [32, 0.00017216801643371582],
 [33, 0.0025776028633117676],
 [34, 0.00049552321434021],
 [35, 0.9850167036056519],
 [36, 0.13257968425750